<img src="https://www.epfl.ch/about/overview/wp-content/uploads/2020/07/logo-epfl-1024x576.png" width="140px" alt="EPFL_logo">

## Image Processing Laboratory Notebooks
---

This Jupyter Notebook is part of a series of computer laboratories that are designed
to teach image-processing programming; they are running on the EPFL's Noto server. They are the practical complement of the theoretical lectures of the EPFL's Master course 
[**MICRO-512 Image Processing II**](https://moodle.epfl.ch/course/view.php?id=463) taught by Prof. M. Unser and Prof. D. Van de Ville.

The project is funded by the Center for Digital Education and the School of Engineering. It is owned by the [Biomedical Imaging Group](http://bigwww.epfl.ch/). 
The distribution or reproduction of the notebook is strictly prohibited without the written consent of the authors.  &copy; EPFL 2025.

**Authors**: 
    [Pol del Aguila Pla](mailto:pol.delaguilapla@epfl.ch), 
    [Kay Lächler](mailto:kay.lachler@epfl.ch),
    [Alejandro Noguerón Arámburu](mailto:alejandro.nogueronaramburu@epfl.ch),
    [Kamil Seghrouchni](mailto:kamil.seghrouchni@epfl.ch), and
    [Daniel Sage](mailto:daniel.sage@epfl.ch).
    

# Lab 4.1: Orientation Warmup
**Released**: Thursday, February 20, 2025

**Submission deadline**: Friday, February 28, 2025, before 23:59 on [Moodle](https://moodle.epfl.ch/course/view.php?id=463)

**Grade weight**: Lab 4 (16 points), 7.5 % of the overall grade

**Help Session**: Thursday, February 27, 2025

**Related lectures**: Chapter 6

Double-click on this cell and fill your name and SCIPER number. Then, run the cell below to verify your identity in Noto and set the seed for random results.

**Student Name**: 

**SCIPER**:

In [ ]:
import getpass
# This line recovers your camipro number to mark the images with your ID
uid = int(getpass.getuser().split('-')[2]) if len(getpass.getuser().split('-')) > 2 else ord(getpass.getuser()[0])
print(f'SCIPER: {uid}')

# Orientation warm-up (3 points)

In this lab, we implement image processing algorithms and systems relying on directional analysis, i.e., on the orientation features of an image.
To obtain orientation features, we mainly use linear filtering, which was covered in [Lab 2: Digital filtering](../2_Filtering_lab/1_Filtering.ipynb) of [Image Processing I](https://moodle.epfl.ch/enrol/index.php?id=522).
In the first part of this lab, we (re-)familiarize ourselves with tools used in the upcoming labs, and discuss some *advanced* and *efficient* filtering techniques.
In particular, we implement efficient approximate Gaussian smoothing, introduced in [Image Processing I](https://moodle.epfl.ch/course/view.php?id=522).
If anything about the basic tools is unclear, make sure to check the introductory lab (Lab 0) again.

# Efficient Gaussian smoothing

Gaussian smoothing is a fundamental part of many image-processing algorithms.
It is often used to denoise images, improving the reliability of downstream algorithms.
We already implemented separable Gaussian smoothing in [Lab 2: Digital Filtering](../2_Filtering_lab/1_Filtering.ipynb), which is provided in the next cell.
Although the separable implementation significantly improves the runtime, it is still not suitable for time-critical applications when $\sigma$ is large.
Therefore, we implement an approximate Gaussian smoothing with runtime independent of $\sigma$.

The next cell implements the separable Gaussian smoothing to compare the execution time.

In [ ]:
import numpy as np


def gaussian(sigma):
    # To capture significant influence of the Gaussian, we consider
    # pixels with up to 3*s distance. + 1 ensures an odd filter size
    # for well-defined center; see also the course notes
    n = int(2 * np.ceil(3 * sigma) + 1)
    x = np.linspace(-n // 2 + 1, n // 2, n)
    g = np.exp(-x ** 2 / (2 * sigma**2))
    return g / g.sum()


def gaussian_filter(image, sigma):
    kernel = gaussian(sigma)
    convolved = np.empty_like(image)

    for i in range(image.shape[0]):
        convolved[i] = filter1d(image[i], kernel)

    for j in range(image.shape[1]):
        convolved[:, j] = filter1d(convolved[:, j], kernel)

    return convolved
    

def filter1d(image, kernel):
    padding = kernel.shape[0] // 2
    padded = np.pad(image, padding, mode='reflect')
    convolved = np.zeros_like(image)
    for i in range(convolved.shape[0]):
        for j in range(kernel.shape[0]):
            convolved[i] += padded[i + j] * kernel[-j - 1]
    return convolved

## Generating the box filter widths (1 point)

We approximate Gaussian smoothing with successive [**box filtering**](https://en.wikipedia.org/wiki/Box_blur) passes with different box widths; theoretical foundations of this are discussed more formally later in the course.
The box kernel is a square with constant values that sum to 1; the box kernel with box width $3$ is

$$\frac{1}{9} \begin{bmatrix} 1 & 1 & 1 \\ 1 & 1 & 1 \\ 1 & 1 & 1 \end{bmatrix}\,.$$

When approximating Gaussian smoothing with successive passes of box filtering, the width of the boxes needs to be chosen to account for $\sigma$ and the number of box filter passes.
In this lab, we explore the choice proposed in [\[1\]](https://www.peterkovesi.com/papers/FastGaussianSmoothing.pdf), though other choices with benefits and drawbacks for different applications exist.
The approximation uses $m$ box filters with width $w_0, w_1, \dots, w_{m-1}$, where

$$w_i = \begin{cases}
    w_0, & \text{if } i < \gamma,\\
    w_0 + 2,              & \text{otherwise,}
\end{cases} \quad \text{where}\ \gamma = \left\lfloor \frac{12 \sigma^2 - mw_0^2 - 4mw_0 - 3 m}{-4w_0 - 4} \right\rceil.$$

There, $w_0$ is defined as

$$w_0 = \begin{cases} 
    \tilde{w}_0\,, &\mbox{if } \tilde{w}_0 \mbox{ is odd} \,, \\
    \tilde{w}_0 - 1\,, & \mbox{ otherwise,} 
\end{cases} \quad\text{and}\ \tilde{w}_0 = \left\lfloor \sqrt{\frac{12 \sigma^2}{m}+1} \right\rfloor.$$


Here, $\lfloor\,\cdot\,\rfloor$ is the floor function (see [`np.floor`](https://numpy.org/doc/stable/reference/generated/numpy.floor.html)) and $\lfloor\,\cdot\,\rceil$ is the rounding function (see [`np.around`](https://numpy.org/doc/stable/reference/generated/numpy.around.html#numpy.around)).

**For 1 point**, implement `box_widths` taking the arguments
 * `sigma`: The standard deviation of the Gaussian blur to approximate,
 * `m`: The number of box filters to use. 
 
and returning:
 * the widths $w_0$ to $w_{m-1}$ in a NumPy array according to the formulas above.

[\[1\]](https://www.peterkovesi.com/papers/FastGaussianSmoothing.pdf): "Fast Almost-Gaussian Filtering," Peter Kovesi, _2010 IEEE International Conference on Digital Image Computing: Techniques and Applications_, Sydney, NSW, Australia

In [ ]:
def box_widths(sigma, m):
    widths = np.empty(m, dtype=np.uint8)
    
    # YOUR CODE HERE
    
    return widths

The next cell performs some sanity checks on `box_widths`.

In [ ]:
refs = {
    (3, 7): np.array([3, 3, 3, 3, 5, 5, 5]),
    (5.4, 9): np.array([5, 5, 5, 7, 7, 7, 7, 7, 7]),
}
ours = {args: box_widths(*args) for args in refs.keys()}

if len(ours[(3, 7)]) != 7:
    print('Warning: The length of the output list should be equal to m.')

if np.any(ours[(3, 7)] % 2 == 0):
    print('Warning: Ensure that the box widths are always odd.')

for args in refs.keys():
    our = ours[args]
    ref = refs[args]
    if not np.allclose(our, ref):
        print(f'Warning: The output for {args} should be {ref}, got {our}.')
    

We compare the box filter approximation to the exactly discretized Gaussian filter and calculate the error between the impulse responses depending on the number of box filters used.
You can change $\sigma$ with the slider below the figure, and you change the maximum number of box passes in the code (now set to $5$).
Observe the effect this has on the MSE.

In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt
import ipywidgets as widgets


def mse_db(a, b):
    length = max(len(a), len(b))
    a_pad = np.pad(a, (length - len(a)) // 2)
    b_pad = np.pad(b, (length - len(b)) // 2)
    return 10*np.log10(np.mean((a_pad - b_pad) ** 2))


sigma_slider = widgets.FloatSlider(value=0.2, min=0.2, max=8, step=0.2, description='\u03C3', continuous_update=False) 
box_passes = 5
ns = np.arange(1, box_passes + 1)

plt.close('all')
fig, axs = plt.subplots(1, 2, figsize=(10, 5))

def plot_approximation(change):
    sigma = change.new
    errors = []
    axs[0].clear()
    axs[1].clear()
    n = int(2 * np.ceil(3 * sigma) + 1)
    x_g = np.linspace(-n // 2 + 1, n // 2, n)
    gauss_exact = gaussian(sigma)
    axs[0].plot(x_g, gauss_exact, 'k')
    # Iterate through box filters
    for p, n in enumerate(ns):
        boxes = box_widths(sigma, n)
        conv = np.ones(boxes[0]) / boxes[0]
        for i in range(1, len(boxes)):
            box = np.ones(boxes[i]) / boxes[i]
            conv = np.convolve(conv, box)
        x_b = np.linspace(-conv.shape[0]//2+1, conv.shape[0]//2, conv.shape[0])
        errors.append(mse_db(conv, gauss_exact))
        axs[0].plot(x_b, conv, alpha=0.5)
    # Format plot
    axs[0].legend(['Exact Gaussian'] + [f'{n} box passes' for n in ns])
    axs[0].set_xlabel('x')
    axs[0].set_ylabel('h[x]')
    axs[0].set_title(f'Gaussian $\\sigma$={sigma:.2f} vs Box filter approximations')
    axs[0].grid()
    axs[1].plot(ns, errors)
    axs[1].set_xlabel('Number of box passes')
    axs[1].set_ylabel('MSE [dB]')
    axs[1].set_title(f'MSE, Gaussian $\\sigma$={sigma:.2f} vs Box filter approximations')
    axs[1].grid()
    fig.tight_layout()

# Didplay widget and link to callback
display(sigma_slider)
sigma_slider.observe(plot_approximation, 'value')
sigma_slider.value = 3

We observe that method does not work well for small $\sigma$, as the box widths are all 1.
However, when $\sigma > 0.6$ the results are acceptable.
We now plot the MSE between the approximation and the exactly discretized Gaussian depending on $\sigma$, with $\sigma$ ranging from $0.6$ to $10$. 
We can change the number of box filtering passes using the slider below the figure.
Observe the effect this has on the MSE.

In [ ]:
# Initialize slider to define the number of box filters to use
box_slider = widgets.IntSlider(value=1, min=1, max=12, step=1, description='Number of box passes', continuous_update=False, 
                               style={'description_width':'initial'}, layout={'width':'400px'}) 

# Declare array with sigma values to evaluate
sigmas = np.linspace(0.5, 10, 1000)

# Initialize Matplotlib figure
plt.close("all")
fig = plt.figure(figsize = (10,6))
ax = plt.gca()

def plot_mse(change):
    n = change.new
    ax.clear()
    errors = []
    for sigma in sigmas:
        gauss_exact = gaussian(sigma)
        boxes = box_widths(sigma, n)
        conv = np.ones(boxes[0]) / boxes[0]
        for i in range(1, len(boxes)):
            box = np.ones(boxes[i]) / boxes[i]
            conv = np.convolve(conv, box)
        errors.append(mse_db(conv, gauss_exact))
    ax.plot(sigmas, errors);
    ax.set_xlabel('$\\sigma$'); ax.set_ylabel('MSE [dB]')
    ax.set_title(f'MSE, {n} box filter approximation vs Gaussian impulse response')
    ax.grid()
    
display(box_slider)
box_slider.observe(plot_mse, 'value')
box_slider.value = 6

## Implementation of a separable accumulation box filter (2 points)

Given the previous analysis, we stick to $m=4$ box filter passes, offering a good compromise between numerical accuracy and execution time. 
We now implement the box filtering efficiently via a **separable accumulator**, making execution time independent of the width of the filter.
In detail, as the box filter is constant, computing the next unknown output amounts to adding the new pixel now covered by the filter and subtracting the pixel no longer covered by the filter.
Thus, the computation time is independent of the filter width, as we only need to perform $2$ operations per pixel (besides the very first pixel in every row/column).
The accumulator needs to be filled only for the first output pixel.

A visual representation of a horizontal accumulation box filter of **width 5** using mirroring boundary conditions applied to the first row is given below.

![Visual representation of the accumulation box filter.](images/box_filter_gif_snapshots.png)

In the next cell, we have implemented `box_gauss_approximation`, which approximates the Gaussian filter through successive passes of box filters with different widths.
Since the box filter is separable, we implement the two-dimensional box filter as two passes of a one-dimensional box filter.
**For 2 points**, implement `box_filter1d` using the accumulator strategy.

In [ ]:
def box_gauss_approximation(img, sigma, m=4):
    output = img.copy()
    widths = box_widths(sigma, m)
    for width in widths:
        output = box_filter2d(output, width=width)
    return output


def box_filter2d(image, width):
    convolved = np.empty_like(image)

    for i in range(image.shape[0]):
        convolved[i] = box_filter1d(image[i], width)

    for j in range(image.shape[1]):
        convolved[:, j] = box_filter1d(convolved[:, j], width)

    return convolved


def box_filter1d(image, width):
    padding = int(width // 2)
    padded = np.pad(image, padding, mode='reflect')
    convolved = np.zeros_like(image)
    # YOUR CODE HERE
    return convolved / width

The next cell has a sanity check on a small example.

In [ ]:
test_img = np.zeros((5, 5))
test_img[0, 0] = 1
test_img[2, 2] = 1
test_img[4, 4] = 1

comp = np.array([
    [1/9, 1/9, 0, 0, 0],
    [1/9, 2/9, 1/9, 1/9, 0],
    [0, 1/9, 1/9, 1/9, 0],
    [0, 1/9, 1/9, 2/9, 1/9],
    [0, 0, 0, 1/9, 1/9]
])

convolved = box_filter2d(test_img, 3)

if not np.allclose(convolved, comp): print('Your function does not pass the sanity check')


# Runtime comparison

Finally, we compare the box filter approximation to the separable Gaussian filter.
First, we compare the runtime of both methods depending on the standard deviation of the Gaussian.
You can experiment with the values, but beware that large sigmas lead to long runtime!
The cell below runs both smoothing methods for all the $\sigma$ values defined in the cell above on the image `dendrochronology`, which has a size of $512 \times 512$ pixels.

In [ ]:
import time
import imageio.v3 as imageio

image = imageio.imread('images/dendrochronolgy.tif') / 255.


sigmas = [0.1, 1, 2, 3, 4, 5, 8]
errors, times_approx, times_exact = [np.empty((len(sigmas), )) for _ in range(3)]
print('Running for s =', end=' ')
for i, sigma in enumerate(sigmas):
    print(f'{sigma:.1f},', end=' ')
    
    start = time.time()
    approximated = box_gauss_approximation(image, sigma)
    times_approx[i] = time.time() - start
    
    start = time.time()
    exact = gaussian_filter(image, sigma)
    times_exact[i] = time.time() - start

print('\nFinished running.')

plt.close('all') 
fig, ax = plt.subplots(1, 2, figsize=(12, 4))
ax[0].plot(sigmas, times_exact, marker='o')
ax[0].plot(sigmas, times_approx, marker='s')
ax[0].legend(['Separable Gaussian', 'Accumulation box filters'])
ax[0].set_xlabel('$\\sigma$')
ax[0].set_ylabel('Runtime [s]')
ax[0].grid()
ax[0].set_title('Runtime comparison')
ax[1].plot(sigmas, times_exact / times_approx, marker='^')
ax[1].set_xlabel('$\\sigma$');
ax[1].grid()
ax[1].set_title('Speedup')
plt.show()

We see that the runtime of the accumulation box filters is independent of $\sigma$, whereas the runtime of the separable Gaussian increases linearly with $\sigma$.
For a non-separable Gaussian filter we would probably still be waiting for the results as the runtime is proportional to $\sigma^2$, demonstrating the benefit of separable filters and the accumulation strategy.

Finally, we compare the accuracy of our method to the Gaussian filter provided by SciPy.

In [ ]:
import scipy.ndimage as nd

sigmas = [0.5, 1, 1.5, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 14, 16, 18, 22, 26, 30]

exact = np.empty((len(sigmas), *image.shape))
approx = np.empty((len(sigmas), *image.shape))
errors = np.empty((len(sigmas), ))
print('Running for \u03C3 =', end=' ')
for i, sigma in enumerate(sigmas):
    print(f"{sigma:.1f},", end=' ')
    approx[i] = box_gauss_approximation(image, sigma)
    exact[i] = nd.gaussian_filter(image, sigma, radius=int(np.ceil(3 * sigma)))
    errors[i] = mse_db(exact[i], approx[i])


print('\nFinished running for all standard deviations.')

# Display result
plt.close('all')
plt.figure('Average pixel error')
plt.plot(sigmas, errors)
plt.xlabel('$\\sigma$')
plt.ylabel('Avg pixel error (%)')
plt.title('Average error per pixel')
plt.grid()
plt.show()

In accordance with the previous analysis, the approximation works well for large standard deviations, getting an MSE below $-50$dB, which is acceptable for most applications.

Finally, run the cell below to visually compare the smoothed images.

In [ ]:
from interactive_kit import imviewer as viewer

# Change the sigma index to see the result for different sigmas
sigma_idx = 11
# Give information to user
print(f'Average pixel error for \u03C3 = {sigmas[sigma_idx]}: {errors[sigma_idx]:.4}%')
plt.close('all')
img_list = [image, exact[sigma_idx], approx[sigma_idx]]
title_list = ['Original Image', f'Gaussian smoothed $\\sigma$={sigmas[sigma_idx]}', f'box filter smoothed $\\sigma$={sigmas[sigma_idx]}']
view = viewer(img_list, title=title_list, subplots=(1,3))

🎉 Congratulations on finishing the first warm-up lab of IP2, reintroducing some of the basic tools we will use later.


Make sure to save your notebook (you might want to keep a copy on your personal computer) and upload it to Moodle, **in a zip file with the other notebook of this lab.**

* Keep the name of the notebook as: *1_orientation_warmup.ipynb*,
* Name the `zip` file: *orientation_lab.zip*.